# Classification 2

We will look at the Naive Bayes classifier.

* Only slightly more complicated than nearest-neighbors in terms of parameter fitting
* Scales to large datasets easily
* Works best when features are _independent_

## Income Prediction

This data was extracted from a census bureau dataset. It contains several features about individuals: their age, working status, education level, marital status, etc. The goal is to predict if the individual makes more or less than 50K per year.

See [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names "Adult names") for details.

In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# We will ignore some silly warnings that pop up due to scikit-learn
import warnings
warnings.filterwarnings('ignore')

In [3]:
names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', \
         'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', \
         'hours_per_week', 'native_country', 'income_band']
df = pd.read_csv('Classification_2_data/adult.data', sep=', ', names=names)
df[:5]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_band
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


#### Create the target

In [5]:
df['target'] = 0.0
mask = (df['income_band'] == '>50K')
df['target'][mask] = 1.0

In [6]:
df['target'].value_counts()

0.0    24720
1.0     7841
Name: target, dtype: int64

### Discretize continuous values

While Naive Bayes can be run with continuous values, we will try it only with categorical features

* **workclass** is categorical; it takes values among
    * Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* **age** is continuous.

For the continuous features, we will _discretize_ them into bins. Thankfully, pandas gives us a simple way to do so.

In [7]:
# NOTE: We also return the bins (retbins=True).
# We will use these bins later on the test set.
df['age_binned'], age_bins = pd.qcut(df['age'], 5, retbins=True)
print(df['age_binned'][:5])

0    (33.0, 41.0]
1    (41.0, 50.0]
2    (33.0, 41.0]
3    (50.0, 90.0]
4    (26.0, 33.0]
Name: age_binned, dtype: category
Categories (5, interval[float64, right]): [(16.999, 26.0] < (26.0, 33.0] < (33.0, 41.0] < (41.0, 50.0] < (50.0, 90.0]]


In [8]:
df['fnlwgt_binned'], fnlwgt_bins = pd.qcut(df['fnlwgt'], 5, retbins=True)
df['education_num_binned'], education_num_bins = pd.qcut(df['education_num'], 3, retbins=True)

Other seemingly continuous features are actually almost categorical. Consider capital_loss, for instance.

In [9]:
df['capital_loss'].value_counts()[:5]

0       31042
1902      202
1977      168
1887      159
1848       51
Name: capital_loss, dtype: int64

In [10]:
len(df)

32561

So, let us bin capital loss as either 0, or greater than 0.

In [11]:
df['capital_loss_binned'] = '0'
df['capital_loss_binned'][df['capital_loss'] > 0] = '>0'

In [12]:
df['capital_gain_binned'] = '0'
df['capital_gain_binned'][df['capital_gain'] > 0] = '>0'

Similarly for hours_per_week: less than 40, 40, and more than 40.

In [13]:
df['hours_binned'] = '40'
df['hours_binned'][df['hours_per_week'] < 40] = '<40'
df['hours_binned'][df['hours_per_week'] > 40] = '>40'

#### Create the design matrices

In [14]:
print(df.columns.values)

['age' 'workclass' 'fnlwgt' 'education' 'education_num' 'marital_status'
 'occupation' 'relationship' 'race' 'sex' 'capital_gain' 'capital_loss'
 'hours_per_week' 'native_country' 'income_band' 'target' 'age_binned'
 'fnlwgt_binned' 'education_num_binned' 'capital_loss_binned'
 'capital_gain_binned' 'hours_binned']


For Naive Bayes, we want all dummy variables, but unfortunately standard patsy doesn't necessarily give us that. So, just for the Naive Bayes classifier, we will use a pandas-specific way of dummy encoding.

In [15]:
categorical_columns = ['age_binned', 'workclass', 'fnlwgt_binned', 'education',
          'education_num_binned', 'marital_status', 'occupation', 'relationship',
          'race', 'sex', 'capital_gain_binned', 'capital_loss_binned', 'hours_binned']
df_dummies = pd.get_dummies(df[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values
dummy_column_names[:10]

array(['age_binned_(16.999, 26.0]', 'age_binned_(26.0, 33.0]',
       'age_binned_(33.0, 41.0]', 'age_binned_(41.0, 50.0]',
       'age_binned_(50.0, 90.0]', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Never-worked',
       'workclass_Private'], dtype=object)

In [16]:
# Concatenate all these new dummy columns into the old dataframe
df2 = pd.concat([df, df_dummies], axis=1)

In [17]:
df2.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,...,race_White,sex_Female,sex_Male,capital_gain_binned_0,capital_gain_binned_>0,capital_loss_binned_0,capital_loss_binned_>0,hours_binned_40,hours_binned_<40,hours_binned_>40
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,...,1,0,1,0,1,1,0,1,0,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,1,0,1,1,0,1,0,0,1,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,...,1,0,1,1,0,1,0,1,0,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,...,0,0,1,1,0,1,0,1,0,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,...,0,1,0,1,0,1,0,1,0,0


In [18]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))
print(formula)

target ~ 0 + Q("age_binned_(16.999, 26.0]") + Q("age_binned_(26.0, 33.0]") + Q("age_binned_(33.0, 41.0]") + Q("age_binned_(41.0, 50.0]") + Q("age_binned_(50.0, 90.0]") + Q("workclass_?") + Q("workclass_Federal-gov") + Q("workclass_Local-gov") + Q("workclass_Never-worked") + Q("workclass_Private") + Q("workclass_Self-emp-inc") + Q("workclass_Self-emp-not-inc") + Q("workclass_State-gov") + Q("workclass_Without-pay") + Q("fnlwgt_binned_(12284.999, 106648.0]") + Q("fnlwgt_binned_(106648.0, 158662.0]") + Q("fnlwgt_binned_(158662.0, 196338.0]") + Q("fnlwgt_binned_(196338.0, 259873.0]") + Q("fnlwgt_binned_(259873.0, 1484705.0]") + Q("education_10th") + Q("education_11th") + Q("education_12th") + Q("education_1st-4th") + Q("education_5th-6th") + Q("education_7th-8th") + Q("education_9th") + Q("education_Assoc-acdm") + Q("education_Assoc-voc") + Q("education_Bachelors") + Q("education_Doctorate") + Q("education_HS-grad") + Q("education_Masters") + Q("education_Preschool") + Q("education_Prof-sc

In [19]:
Y, X = dmatrices(formula, df2, return_type='dataframe')

In [20]:
y = Y['target'].values
y[:10]

array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1.])

#### Set up the classifier

In [21]:
from sklearn import naive_bayes
model = naive_bayes.MultinomialNB()

#### Fit the model

In [22]:
model.fit(X, y)

MultinomialNB()

#### Test on some training data

In [23]:
print('Prediction')
print(model.predict(X[:10]))
print('Actual')
print(y[:10])

Prediction
[0. 1. 0. 0. 1. 1. 0. 1. 1. 1.]
Actual
[0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]


In [24]:
from sklearn import metrics

prediction_train = model.predict(X)
print(metrics.accuracy_score(y, prediction_train))

0.8188937686189


#### Test

We load the test data from another CSV file.

In [25]:
df_test = pd.read_csv('Classification_2_data/adult.test', sep=', ', names=names)

#### Create the target

In [26]:
df_test['target'] = 0.0
mask = (df_test['income_band'] == '>50K')
df_test['target'][mask] = 1.0

#### Do the feature binning

In [27]:
# Use the bins we created using from the training set
# Now, we need pd.cut instead of pd.qcut
df_test['age_binned'] = pd.cut(df_test['age'], bins=age_bins, include_lowest=True)
df_test['fnlwgt_binned'] = pd.cut(df_test['fnlwgt'], bins=fnlwgt_bins, include_lowest=True)
df_test['education_num_binned'] = pd.cut(df_test['education_num'], bins=education_num_bins, include_lowest=True)

df_test['capital_loss_binned'] = '0'
df_test['capital_loss_binned'][df_test['capital_loss'] > 0] = '>0'

df_test['capital_gain_binned'] = '0'
df_test['capital_gain_binned'][df_test['capital_gain'] > 0] = '>0'

df_test['hours_binned'] = '40'
df_test['hours_binned'][df_test['hours_per_week'] < 40] = '<40'
df_test['hours_binned'][df_test['hours_per_week'] > 40] = '>40'

#### Create the design matrices

In [28]:
df_dummies = pd.get_dummies(df_test[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = df_dummies.columns.values
df_test2 = pd.concat([df_test, df_dummies], axis=1)
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))

In [29]:
Y_test, X_test = dmatrices(formula, df_test2, return_type='dataframe')
y_test = Y_test['target'].values

#### Run the test

In [30]:
prediction_test = model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction_test))

0.8220625268718138


* Testing accuracy was similar to training accuracy
    * The accuracy on the training set was 0.819
* This is good; it means we are not overfitting
    * With overfitting, we would have great training accuracy,
    * but the classifier would perform poorly on the test set.

#### Class priors

* Class priors are encoded in the "model" variable, after fitting the model to the training data. 
* The priors are available from *model.class_log_prior_*
    * This is not the fraction of each class in the training data, but rather
    * the _logarithm_ of that.

In [31]:
print('Prior probability for the negative class is',)
print(exp(model.class_log_prior_[0]))
print('Prior probability for the positive class is',)
print(exp(model.class_log_prior_[1]))

Prior probability for the negative class is
0.7591904425539763
Prior probability for the positive class is
0.24080955744602456


This says:

* first class (class 0) has prior probability 0.76
* second class (class 1) has probability 0.24

We can quickly check that this is exactly the fraction of these classes in the training data.

In [32]:
df['target'].value_counts() / len(df)

0.0    0.75919
1.0    0.24081
Name: target, dtype: float64

This also says that we can easily achieve 76% accuracy: just predict negative class always.

Our accuracy of 82% is better than this baseline, but perhaps not by a lot.

#### What are the likelihoods?

There is a special attribute called **feature\_log\_prob\_** that is available in the _model_ variable after fitting the training data.

In [33]:
# Log of the likelihoods for class 1 (positive class) for the first 5 features
model.feature_log_prob_[1][:5]

array([-6.35107198, -4.55837662, -3.87227007, -3.75926107, -3.88363922])

In [34]:
print('Likelihoods for the negative class (first 5 features)')
print(np.exp(model.feature_log_prob_[0])[:5])
print('Likelihoods for the positive class (first 5 features)')
print(np.exp(model.feature_log_prob_[1])[:5])

Likelihoods for the negative class (first 5 features)
[0.02183922 0.0152439  0.01444126 0.0118218  0.0135733 ]
Likelihoods for the positive class (first 5 features)
[0.00174488 0.01047906 0.02081107 0.02330095 0.02057581]


#### Which features are most important?

A feature is important (i.e. very **discriminative**) if it is very _likely_ in one class but very _unlikely_ in the other.

$$\mbox{If }\frac{P(\mbox{feature in positive class})}{P(\mbox{feature in negative class})} \gg 1 \Rightarrow \mbox{presence of feature implies positive class}$$\


$$\mbox{If }\frac{P(\mbox{feature in positive class})}{P(\mbox{feature in negative class})} \ll 1 \Rightarrow \mbox{presence of feature implies negative class}$$

$$\mbox{In either case,} \log\left(\frac{P(\mbox{feature in positive class})}{P(\mbox{feature in negative class})}\right) \mbox{is very large in magnitude}$$

In [35]:
feature_importances = abs(model.feature_log_prob_[1] - model.feature_log_prob_[0])
feature_importances

array([2.5270243 , 0.37480093, 0.36539552, 0.67854877, 0.41601179,
       1.00089736, 0.68648196, 0.27643082, 0.9317309 , 0.12556119,
       1.3776994 , 0.22839003, 0.16476499, 1.56033956, 0.03601936,
       0.09083016, 0.06662479, 0.07646216, 0.05127959, 1.47994405,
       1.75892163, 1.31989026, 2.00012941, 1.78112739, 1.54724608,
       1.71040025, 0.04285224, 0.10983809, 0.80348916, 2.19242717,
       0.51369456, 1.3747681 , 2.80353307, 2.1604915 , 0.30022494,
       0.75221789, 0.30022494, 0.87100125, 1.00169313, 0.90654859,
       0.9343262 , 1.25018463, 1.88326713, 1.51453002, 1.21028715,
       1.00514108, 0.71251662, 0.35636675, 0.07890704, 1.0837754 ,
       0.87862107, 1.54489523, 0.7959203 , 1.98304647, 3.16308848,
       0.94323488, 0.41979751, 0.15025213, 0.32588538, 0.23389987,
       0.9412969 , 1.01482075, 2.06588812, 3.15037476, 1.5431454 ,
       1.04826572, 0.86177231, 0.13316036, 0.80626702, 1.10358115,
       0.08020373, 0.94780858, 0.32766705, 0.19828537, 1.62994

We have the feature weights, but what are the names of these features?

Recall that the features are the columns of the _design matrix_ X, so let's put all the information together in a Series.

In [36]:
feature_importance_series = Series(feature_importances, index=X.columns.values)
feature_importance_series[:10]

Q("age_binned_(16.999, 26.0]")    2.527024
Q("age_binned_(26.0, 33.0]")      0.374801
Q("age_binned_(33.0, 41.0]")      0.365396
Q("age_binned_(41.0, 50.0]")      0.678549
Q("age_binned_(50.0, 90.0]")      0.416012
Q("workclass_?")                  1.000897
Q("workclass_Federal-gov")        0.686482
Q("workclass_Local-gov")          0.276431
Q("workclass_Never-worked")       0.931731
Q("workclass_Private")            0.125561
dtype: float64

How can we find the top features?

In [37]:
feature_importance_series.sort_values(ascending=False)[:10]

Q("occupation_Priv-house-serv")      3.163088
Q("relationship_Own-child")          3.150375
Q("education_Preschool")             2.803533
Q("age_binned_(16.999, 26.0]")       2.527024
Q("education_Doctorate")             2.192427
Q("education_Prof-school")           2.160491
Q("relationship_Other-relative")     2.065888
Q("education_1st-4th")               2.000129
Q("occupation_Other-service")        1.983046
Q("marital_status_Never-married")    1.883267
dtype: float64

These are the absolute differences

* absolute value of log P(positive class) - log P(negative class).

But how do we figure out if the numbers were

* positive (i.e., higher probability in positive class), or
* negative (i.e., higher probability in negative class)?

In [38]:
top_10_feature_indices = feature_importance_series.sort_values(ascending=False)[:10].index.values

In [39]:
inter_class_differences = model.feature_log_prob_[1] - model.feature_log_prob_[0]
new_feature_importance_series = Series(inter_class_differences, index=X.columns.values)

new_feature_importance_series.loc[top_10_feature_indices]

Q("occupation_Priv-house-serv")     -3.163088
Q("relationship_Own-child")         -3.150375
Q("education_Preschool")            -2.803533
Q("age_binned_(16.999, 26.0]")      -2.527024
Q("education_Doctorate")             2.192427
Q("education_Prof-school")           2.160491
Q("relationship_Other-relative")    -2.065888
Q("education_1st-4th")              -2.000129
Q("occupation_Other-service")       -1.983046
Q("marital_status_Never-married")   -1.883267
dtype: float64